# Automated EDA on the titanic dataset

In [1]:
!pip install ydata-profiling pyarrow awscli s3fs -q

In [2]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

In [3]:
import os

storage_options={
    'key': os.environ['AWS_ACCESS_KEY_ID'],
    'secret': os.environ['AWS_SECRET_ACCESS_KEY'],
    'client_kwargs':{
        'endpoint_url': os.environ['MINIO_ENDPOINT_URL']
    }
}

## Load data

In [4]:
!wget https://raw.githubusercontent.com/choas/h2o-titanic/master/data/train.csv

--2023-03-05 11:22:25--  https://raw.githubusercontent.com/choas/h2o-titanic/master/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61194 (60K) [text/plain]
Saving to: ‘train.csv.2’

train.csv.2         100%[===================>]  59.76K  --.-KB/s    in 0.01s   

2023-03-05 11:22:25 (4.98 MB/s) - ‘train.csv.2’ saved [61194/61194]



In [5]:
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
profile = ProfileReport(df, title="Profiling Report")

In [7]:
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
nrows_prev = len(df)

df_clean = df[~df["Embarked"].isna()]
df_clean = df[~df["Age"].isna()]

print("%d rows removed" % (nrows_prev - len(df_clean)))

177 rows removed


In [9]:
df_clean.to_parquet("s3://bpk/titanic_cleaned.parquet", storage_options=storage_options)

In [10]:
!aws --endpoint-url $MINIO_ENDPOINT_URL s3 ls s3://bpk/

2023-03-05 11:22:39      41352 titanic_cleaned.parquet
